# Notebook for exporting screenshots for annotation

In [49]:
import os
import json
import shutil

from ultralytics import YOLO
from PIL import Image

import sys
sys.path.insert(0, "../../hindsight/hindsight_server/")

from db import HindsightDB
from utils import make_dir

In [50]:
db = HindsightDB()

In [51]:
frames = db.get_frames()

In [53]:
frames = frames.loc[frames['application'] == "Twitter"]

In [54]:
sel_frames = frames.sample(n=100)

In [55]:
dest_dir = "../data/annotations/twitter/rand_upload_2/"
make_dir(dest_dir)
for f in sel_frames['path']:
    dest_f = os.path.join(dest_dir, os.path.basename(f))
    shutil.copy(f, dest_dir)

# Run prediction model on frames

In [11]:
trained_model = YOLO("./runs/detect/train15/weights/best.pt")

In [42]:
pred_images_dir = dest_dir 
images = list()
for f in os.listdir(pred_images_dir):
    im = Image.open(os.path.join(pred_images_dir, f))
    images.append(im)

In [44]:
results = trained_model(images)


0: 640x320 1 more_posted, 4 tweets, 153.3ms
1: 640x320 5 tweets, 153.3ms
2: 640x320 1 more_posted, 3 tweets, 153.3ms
3: 640x320 3 tweets, 153.3ms
4: 640x320 3 tweets, 153.3ms
5: 640x320 6 tweets, 153.3ms
6: 640x320 1 more_posted, 2 tweets, 153.3ms
7: 640x320 1 new_posts, 7 tweets, 1 twitter_top_menu, 153.3ms
8: 640x320 3 tweets, 153.3ms
9: 640x320 1 more_posted, 3 tweets, 153.3ms
10: 640x320 5 tweets, 153.3ms
11: 640x320 2 tweets, 153.3ms
12: 640x320 4 tweets, 1 tweet_ad, 153.3ms
13: 640x320 3 tweets, 153.3ms
14: 640x320 5 tweets, 153.3ms
15: 640x320 1 more_posted, 3 tweets, 153.3ms
16: 640x320 1 more_posted, 3 tweets, 153.3ms
17: 640x320 2 tweets, 153.3ms
18: 640x320 1 more_posted, 3 tweets, 153.3ms
19: 640x320 5 tweets, 1 twitter_top_menu, 153.3ms
20: 640x320 1 plus_button, 2 tweets, 1 twitter_bottom_menu, 153.3ms
21: 640x320 3 tweets, 153.3ms
22: 640x320 1 more_posted, 3 tweets, 153.3ms
23: 640x320 1 more_posted, 1 plus_button, 3 tweets, 1 twitter_bottom_menu, 1 twitter_top_menu, 1

In [45]:
all_preds = list()

In [47]:
for result in results:
    image_preds_d = {}
    image_path = f"/data/local-files/?d={result.path}"
    image_preds_d['data'] = {"image" : image_path}
    predictions_d = {"model_version": "train13", "score": 0.55}
    org_width = result.orig_shape[1]
    org_height = result.orig_shape[0]
    result_d_template = {"type": "rectanglelabels",        
            "from_name": "label", "to_name": "image",
            "original_width": org_width, "original_height": org_height,
            "image_rotation": 0}
    converted_results = list()
    for i, box in enumerate(result.boxes):
        result_d = result_d_template.copy()
        result_d['id'] = f"result{i}"
        value_d = {"rotation" : 0,
                "x" : ((float(box.xyxyn[0][0])) * 100),
                "y" : (float(box.xyxyn[0][1])) * 100,
                "width" : ((float(box.xyxyn[0][2]) - float(box.xyxyn[0][0])) * 100), 
                "height": (float(box.xyxyn[0][3]) - float(box.xyxyn[0][1])) * 100, 
                "rectanglelabels": [result.names[int(box.cls[0])]]}
        result_d['value'] = value_d
        converted_results.append(result_d)
        
    predictions_d["result"] = converted_results
    image_preds_d['predictions'] = [predictions_d]
    all_preds.append(image_preds_d)

In [48]:
with open("first_local_preds.json", 'w') as outfile:
    json.dump(all_preds, outfile)